In [ ]:
import libsbml
import importlib
import amici
import amici.plotting
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import scipy.stats

from gm import gm
from gm_Prep import gm_Prep

In [ ]:
# np.random.seed(1337)

In [ ]:
# SBML model we want to import
sbml_file = 'SPARCED_Brep.xml'
# Name of the model that will also be the name of the python module
model_name = sbml_file[0:-4]
# Directory to which the generated model code is written
model_output_dir = model_name

spdata0 = pd.read_csv('Species_Brep.txt',header=0,index_col=0,sep="\t")
spdata = np.double(spdata0.values[:,1])

spVolNames = spdata0.values[:,0]
spVols = np.zeros(shape=(774,1))
spVols[np.argwhere(spVolNames=='Cytoplasm')] = 5.2500E-12
spVols[np.argwhere(spVolNames=='Nucleus')] = 1.7500E-12
spVols[np.argwhere(spVolNames=='Mitochondrion')] = 3.6750E-13
spVols[np.argwhere(spVolNames=='Extracellular')] = 5.0000E-05

kGsRead = pd.read_csv('OmicsData.txt',header=0,index_col=0,sep="\t")
gExp_mpc = np.float64(kGsRead.values[:,0])
mExp_mpc = np.float64(kGsRead.values[:,1])
kGin = np.float64(kGsRead.values[:,2])
kGac = np.float64(kGsRead.values[:,3])
kTCleak = np.float64(kGsRead.values[:,4])
kTCmaxs = np.float64(kGsRead.values[:,5])
kTCd = np.float64(kGsRead.values[:,6])
kmRNAs = kGsRead.values[:,7]

# Read-in the activators matrix and assign concentrations of activators
TAsRead = pd.read_csv('TAs.csv',header=0,index_col=0)
TAs0 = np.float64(TAsRead.values)
# Read-in the repressors matrix and assign concentrations of repressors
TRsRead = pd.read_csv('TRs.csv',header=0,index_col=0)
TRs0 = np.float64(TRsRead.values)

In [ ]:
genedata, GenePositionMatrix, AllGenesVec, xgac_mpc_D, xgin_mpc_D, xgac_mpc, xgin_mpc, kTCleak2 \
= gm_Prep(flagD, gExp_mpc, mExp_mpc, kGin, kGac, kTCleak, kTCmaxs, kTCd)

In [ ]:
flagD = 1
th = 24
nmxlsfile = 'FigS2D_det_24hrs_'
ligConcs = [100.0,100.0,100.0,100.0,100.0,100.0,100.0]
ligandInds = [155,156,157,158,159,160,161]

ts = 30
NSteps = th*3600/ts
NSteps = int(NSteps)
Vn = 1.75E-12
Vc = 5.25E-12
tout_all = np.arange(0,th*3600+1,30)

spdata2 = spdata.copy()
spdata2[ligandInds] = ligConcs   
xoutS_all = np.zeros(shape=(NSteps+1,len(spdata2)))
xoutS_all[0,:] = spdata2

sys.path.insert(0, os.path.abspath(model_output_dir))
model_module = importlib.import_module(model_name)
model = model_module.getModel()
solver = model.getSolver() # Create solver instance
solver.setMaxSteps = 1e10
model.setTimepoints(np.linspace(0,ts)) # np.linspace(0, 30) # set timepoints

startTime = datetime.now()
print(startTime)

conds = []
for qq in range(NSteps):
    genedata,AllGenesVec = gm(flagD,ts,genedata,xoutS_all[qq,:],Vn,Vc,kGin,kGac, \
                               kTCmaxs,kTCleak,kTCd,AllGenesVec,GenePositionMatrix,TAs0,TRs0)
    for ii,kk in enumerate(kmRNAs):
        model.setFixedParameterById(kk,genedata[ii+282]*(1E9/(Vc*6.023E+23)))
    model.setInitialStates(xoutS_all[qq,:])
    rdata = amici.runAmiciSimulation(model, solver)  # Run simulation
    xoutS_all[qq+1,:] = rdata['x'][-1,:]
    if rdata['x'][-1,103] < rdata['x'][-1,105]:
        print('Apoptosis happened')
        break
xoutS_all = xoutS_all[~np.all(xoutS_all == 0, axis=1)]
condsSDF = pd.DataFrame(data=xoutS_all,columns=[ele for ele in model.getStateIds()]) 
condsSDF.to_csv(nmxlsfile+'S.txt',sep="\t")    
condsSDF = None
print(datetime.now())

In [ ]:
mpc2nmcf_Vc = 1.0E9/(Vc*6.023E+23)
plt.figure(figsize=(7, 4))
yy = np.array(condsDF[:,0]*(1/mpc2nmcf_Vc))
tt = (np.arange(0,len(condsDF))*30.0/3600.0)
plt.plot(tt, yy,linewidth=2)
plt.xlim([0, 24])
plt.savefig('FigS2D_v1.png')